In [7]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging
import os
import glob

In [8]:
# Get all files in working data directory
path = os.getcwd()
path = path.replace("notebooks","data")
csv_files = glob.glob(os.path.join(path,"*.csv"))

c:\Github Repo\Mario-Jupyter-Codespace\data\basketballProject


In [9]:
# bringing in all new files
full_df = pd.DataFrame()
for file in csv_files:
    df = pd.read_csv(file)
    frames = [full_df, df]
    full_df = pd.concat(frames)

In [10]:
# Cosmetic changes and dropping duplicaes that snuck in on import
full_df = full_df.rename(columns={"Unnamed= 0": "Season #"})
full_df = full_df.drop_duplicates()

In [12]:
# # MWAR Formula Coefficients:
# coFGM = 2
# coFGA = -1.2
# coFG3M = 1
# coFG3A = -0.4
# coFTM = 1
# coFTA = -1
# coDREB = 1.5
# coOREB = 2
# coAST = 2.5
# coTOV = -3
# coSTL = 2
# coBLK = 2
# coPF = -0.5
# coPTS = 1 

# totFGM = coFGM * full_df["FGM"]
# totFGA = coFGA * full_df["FGA"]
# totFG3M = coFG3M * full_df["FG3M"]
# totFG3A = coFG3A * full_df["FG3A"]
# totFTM = coFTM * full_df["FTM"]
# totFTA = coFTA * full_df["FTA"]
# totDREB = coDREB * full_df["DREB"]
# totOREB = coOREB * full_df["OREB"]
# totAST = coAST * full_df["AST"]
# totTOV = coTOV * full_df["TOV"]
# totSTL = coSTL * full_df["STL"]
# totBLK = coBLK * full_df["BLK"]
# totPF = coPF * full_df["PF"]
# totPTS = coPTS * full_df["PTS"]
# totMIN = full_df["MIN"] - 18

# full_df["MWAR"] = totFGM + totFGA + totFG3A + totFG3M + totFTM + totFTA + totDREB + totOREB + totAST + totTOV + totSTL + totBLK + totPF + totPTS + totMIN

In [41]:
FGMA = (full_df["FGA"] - full_df["FGM"])*1.25
FG3MA = (full_df["FG3A"] - full_df["FG3M"])*1.25
FTMA = (full_df["FTA"]-full_df["FTM"])*0.9
REB = 3*full_df["OREB"] + 2*full_df["DREB"]
ATR = 3*(full_df["AST"]-full_df["TOV"])
STB = 2*(full_df["BLK"]+full_df["STL"])
PF = full_df["PF"]*-0.5
PTS = full_df["PTS"]*1
MIN = full_df["MIN"] - 18*full_df["GP"]

full_df["MOWAR"] = FGMA + FG3MA + FTMA + PTS
full_df["MDWAR"] = STB + PF
full_df["MPWAR"] = ATR
full_df["MIWAR"] = REB + MIN
full_df["MWAR"] = FGMA + FG3MA + FTMA + REB + ATR + STB + PF + PTS + MIN
full_df["MWARPM"] = full_df["MWAR"]/full_df["MIN"]
full_df["MWARPG"] = full_df["MWAR"]/full_df["GP"]

In [49]:
# full_2021_test = full_df.iloc[:,[2,3,8,-8,-7,-6,-5,-4,-3,-2,-1]][(full_df["SEASON_ID"] == "2022-23") & (full_df["GP"] > 20)].sort_values("MWARPM", ascending = False)